# Estimate Ingredient Quantites from Nutrient Info

Given a food product with an FDA label listing $d$ nutrient quantities (e.g. Calories, carbohydrates, etc.) and $p$ ingredients ordered by weight, we calculate the quantity of each ingredient in the product.

We take a constrained optimization approach, seeking the optimal solution $x$ to the linear system $Ax=b$, where $A$ is a matrix describing the nutrient contents of the ingredients, $b$ is a vector listing the total nutrient contents of the products, and $x$ is the quantity of each ingredient.

The system is subject to the following constraints:
- All ingredient quantities are positive ($x>0$)
- The ingredient quantities are ordered ($x_{i+1}>x_i$) as given in the ingredients list.

**NOTE:** When the number of ingredients is greater than the number of nutrients on the FDA label (when $p>d$), there is no unique solution, and further analysis is required to construct bounds for the nutrient quantities. 

## Imports

In [123]:
import numpy as np
import scipy.optimize as optim
import matplotlib.pyplot as plt

In [124]:
%matplotlib inline

## Synthesize Data

In [125]:
def gen_data(d, p):
    """
    d = Number of FDA requirements (known)
    p = Number of ingredients
    """
    
    # Coefficient matrix:
    # FDA nutrient info for each ingredient
    # - each column is an ingredient
    # - FDA nutrient varies over rows
    # - Sorted in order of increasing quantity (first is smallest)
    A = np.random.rand(d, p)

    # Quantities of each ingredient, sorted in increasing order.
    # (correct solution)
    x = np.sort(np.random.rand(p))

    # Sum FDA nutrients of the product
    b = A@x
    
    return A, x, b

## Solve optimization problem

In [126]:
def estim_quantities(A, b, tol=None):
    # Objective function
    def obj(x):
        return np.sum((A@x-b)**2)

    # Ordering constraint matrix
    # each row: c_i^T * x >= 0
    C = np.zeros([p, p])
    # Require that each value is larger than the previous
    for i in range(p):
        for j in range(p):
            if i == j:
                C[i,j] = 1
            elif j == i-1:
                C[i,j] = -1

    # non-negative constraint (0 <= c_i^T *x <= inf)
    lin_const = optim.LinearConstraint(C, 0, np.inf, keep_feasible=False)
    
    # Solve!
    result = optim.minimize(obj, np.zeros(p), method="COBYLA", constraints=lin_const, tol=tol)

    return result

## Apply some noise to simulate real data

In [127]:
def apply_noise(A, z):
    return A * (1.0 + z * 2*(np.random.rand(*np.shape(A))-0.5))

## Test it out!

In [128]:
# Number of FDA requirements (known)
d = 10
# Number of ingredients
p = 5
# Relative noise to apply to A (reported FDA vals. for ingredients)
zA = 0.05
# Relative noise to apply to b (reported FDA vals. for product)
zb = 0.05

# Generate data & apply noise
A, x, b = gen_data(d, p)
A_noise = apply_noise(A, zA)
b_noise = apply_noise(b, zb)

#print("x = {}".format(x))
print()

print("Without noise:")
res = estim_quantities(A, b)
x_star = res.x
#print("x* = {}".format(x_star))
print("rel. err. = {:.2e}".format(np.linalg.norm(x-x_star)/np.linalg.norm(x)))
print()

print("With noise (zA={:.2e}, zb={:.2e}):".format(zA, zb))
res = estim_quantities(A_noise, b_noise)
x_star = res.x
#print("x_n* = {}".format(x_star))
print("rel. err. = {:.2e}".format(np.linalg.norm(x-x_star)/np.linalg.norm(x)))


Without noise:
rel. err. = 1.22e-03

With noise (zA=5.00e-02, zb=5.00e-02):
rel. err. = 5.21e-02
